In [ ]:
import os
os.chdir("../")

from detectmatelibrary.detectors.new_value_combo_detector import NewValueComboDetector, schemas
from detectmatelibrary.parsers.template_matcher import MatcherParser
from detectmatelibrary.readers.log_file import LogFileReader

from detectmatelibrary.common.persistency.event_data_structures.trackers import (
    EventVariableTracker, StabilityTracker
)
from detectmatelibrary.common.persistency.event_data_structures.dataframes import (
    EventDataFrame, ChunkedEventDataFrame
)
from detectmatelibrary.common.persistency.event_persistency import EventPersistency

import logging
logging.getLogger().setLevel(logging.ERROR)  # Only show errors

In [8]:
import yaml


with open("config/pipeline_config_default.yaml", 'r') as f:
    config = yaml.safe_load(f)

In [9]:
reader = LogFileReader(config=config)
parser = MatcherParser(config=config)
detector = NewValueComboDetector(config=config)

output = schemas.DetectorSchema()

In [10]:
reader.reset()
for i in range(1000):
    log = reader.process(as_bytes=False)
    parsed_log = parser.process(log)
    detector.configure(parsed_log)

detector.set_configuration()

In [11]:
reader.reset()
for i in range(1000):
    log = reader.process(as_bytes=False)
    parsed_log = parser.process(log)
    detector.train(parsed_log)

In [12]:
predictions = {}

for i in range(1000):
    log = reader.process(as_bytes=False)
    parsed_log = parser.process(log)
    prediction = detector.detect(parsed_log, output_=output)
    predictions[i] = prediction


In [13]:
# get true predictions
[(i, is_anomaly) for i, is_anomaly in predictions.items() if is_anomaly]

[(859, True),
 (860, True),
 (861, True),
 (862, True),
 (864, True),
 (865, True),
 (866, True),
 (867, True)]